# ***YOLOv12 + EfficientNetB0 + BiFPN***

In [1]:
# Install required packages (for Google Colab)
!pip install -q ultralytics roboflow

import os
from pathlib import Path
import yaml
import torch
import torch.nn as nn
import inspect
import re
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.nn import tasks

# Custom BiFPN Concat module
class BiFPN_Concat2(nn.Module):
    def __init__(self, dimension=1):
        super(BiFPN_Concat2, self).__init__()
        self.d = dimension
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 0.0001

    def forward(self, x):
        w = self.w
        weight = w / (torch.sum(w, dim=0) + self.epsilon)
        x = [weight[0] * x[0], weight[1] * x[1]]
        return torch.cat(x, self.d)

# Register custom module
tasks.BiFPN_Concat2 = BiFPN_Concat2

# Patch parse_model to handle BiFPN_Concat2 like Concat (with indentation preservation)
source = inspect.getsource(tasks.parse_model)
pattern = r'(\s*)else:\s+c2\s*=\s*ch\s*\[f\]'
replacement = r'\1elif m is BiFPN_Concat2:\n\1 c2 = sum([ch[x] for x in f])\n\1else:\n\1 c2 = ch[f]'
modified_source = re.sub(pattern, replacement, source)
exec(modified_source, tasks.__dict__)
print("✅ Custom BiFPN_Concat2 registered and parse_model patched with fixed indentation")

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 10

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# YOLO12 YAML configuration with EfficientNetB0 backbone and BiFPN neck
yolo12_yaml_content = """
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

# EfficientNetB0 backbone via TorchVision (adapted to match YOLOv12 outputs)
backbone:
  # [from, repeats, module, args]
  - [-1, 1, TorchVision, [0, efficientnet_b0, DEFAULT, True, 2, True]]  # 0: features
  - [0, 1, Index, [40, 4]]  # 1: P3 raw (40ch, /8)
  - [1, 1, Conv, [256, 1, 1]]  # 2: P3 projected (256ch, /8)
  - [0, 1, Index, [112, 6]]  # 3: P4 raw (112ch, /16)
  - [3, 1, Conv, [512, 1, 1]]  # 4: P4 projected (512ch, /16)
  - [0, 1, Index, [320, 8]]  # 5: P5 raw (320ch, /32)
  - [5, 1, Conv, [1024, 1, 1]]  # 6: P5 projected (1024ch, /32)

# BiFPN neck (modified from YOLO12 head with weighted fusion)
head:
  - [6, 1, nn.Upsample, [None, 2, 'nearest']]  # 7: up P5
  - [[-1, 4], 1, BiFPN_Concat2, [1]]  # 8: weighted cat up P5 + P4 projected
  - [8, 1, Conv, [512, 1, 1]]  # 9: project to 512ch
  - [9, 1, nn.Upsample, [None, 2, 'nearest']]  # 10: up projected P4
  - [[-1, 2], 1, BiFPN_Concat2, [1]]  # 11: weighted cat up P4 + P3 projected
  - [11, 1, Conv, [256, 1, 1]]  # 12: project to 256ch (P3 refined)
  - [12, 1, Conv, [256, 3, 2]]  # 13: down P3 to P4
  - [[-1, 9], 1, BiFPN_Concat2, [1]]  # 14: weighted cat down P3 + P4 top
  - [14, 1, Conv, [512, 1, 1]]  # 15: project to 512ch (P4 refined)
  - [15, 1, Conv, [512, 3, 2]]  # 16: down P4 to P5
  - [[-1, 6], 1, BiFPN_Concat2, [1]]  # 17: weighted cat down P4 + P5 projected
  - [17, 1, Conv, [1024, 1, 1]]  # 18: project to 1024ch (P5 refined)
  - [[12, 15, 18], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

# Save YOLO12 YAML to file
yaml_path = Path("/content/yolo12n_efficientnetb0_bifpn.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(yaml.safe_load(yolo12_yaml_content), f)
print(f"✅ YAML model saved: {yaml_path}")

# Inisialisasi model
model = YOLO(str(yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_effnetb0_bifpn", epochs=50):
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_effnetb0_bifpn/weights/best.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 107.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Custom BiFPN_Concat2 registered and parse_model patched with fixed indentation
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dental-caries-10 in yolov12:: 100%|██████████| 16480/16480 [00:03<00:00, 4471.15it/s]


✅ Dataset berhasil diunduh: /content/dental-caries-10/data.yaml
Struktur dataset: ['README.dataset.txt', 'README.roboflow.txt', 'data.yaml', 'test', 'train', 'valid']
✅ YAML model saved: /content/yolo12n_efficientnetb0_bifpn.yaml
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 121MB/s]


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
